## Main funtion

In [6]:
import tensorflow as tf
import cv2
import math
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import mysql.connector

label_dict_s = {0: 'default', 1: 'fire', 2: 'smoke'}
IMG_SIZE = 224
new_model = tf.keras.models.load_model('my_model')
device_id = ""

def predict(  model, video_path, filename, label_dict ):
    
    vs = cv2.VideoCapture(video_path)
    fps = math.floor(vs.get(cv2.CAP_PROP_FPS))
    ret_val = True
    writer = 0
    
    while True:
        ret_val, frame = vs.read()
        if not ret_val:
            break
       
        resized_frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame_for_pred = prepare_image_for_prediction( resized_frame )
        pred_vec = model.predict(frame_for_pred)
        #print(pred_vec)
        pred_class =[]
        confidence = np.round(pred_vec.max(),2) 
        
        if confidence > 0.5:
            pc = pred_vec.argmax()
            pred_class.append( (pc, confidence) )
        else:
            pred_class.append( (0, 0) )
        if pred_class:
            txt = get_display_string(pred_class, label_dict)       
            frame = draw_prediction( frame, txt )
        #print(pred_class)
        #plt.axis('off')
        #plt.imshow(frame)
        #plt.show()
        #clear_output(wait = True)
        if not writer:
            fourcc = cv2.VideoWriter_fourcc(*"XVID")
            writer = cv2.VideoWriter(filename, fourcc, fps,(frame.shape[1], frame.shape[0]), True)
            
        # write the out
        writer.write(frame)
        
    vs.release()
    writer.release()

def predict_cam(  model, label_dict, device_id , vid_source = 0):
    
    vs = cv2.VideoCapture(vid_source)
    fps = math.floor(vs.get(cv2.CAP_PROP_FPS))
    ret_val = True
    writer = 0
    count = 0
    arr = []
    
    while True:
        ret_val, frame = vs.read()
        count += 1
        if not ret_val:
            break
       
        resized_frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame_for_pred = prepare_image_for_prediction( resized_frame )
        pred_vec = model.predict(frame_for_pred)
        #print(pred_vec)
        pred_class =[]
        confidence = np.round(pred_vec.max(),2) 
        
        if confidence > 0.5:
            pc = pred_vec.argmax()
            pred_class.append( (pc, confidence) )
        else:
            pred_class.append( (0, 0) )
        if pred_class:
            txt = get_display_string(pred_class, label_dict)       
            frame = draw_prediction( frame, txt )
            if count < 30:
                (c, confidence) = pred_class[-1]
                arr.append(c)
            elif count == 30:
                arr_new = np.array(arr)
                flag_fire = np.bincount(arr_new).argmax()
                if flag_fire == 1:
#                     print("fire detected")
                    findMails(device_id)
                count = 0
                flag_fire = 0
        frame = cv2.resize(frame, (1000,600))
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1)
        if key == 27:
            break 
    vs.release()
    cv2.destroyAllWindows()

def prepare_image_for_prediction( img):
   
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    # The below function inserts an additional dimension at the axis position provided
    img = np.expand_dims(img, axis=0)
    # perform pre-processing that was done when resnet model was trained.
    return preprocess_input(img)

def get_display_string(pred_class, label_dict):
    txt = ""
    for c, confidence in pred_class:
        txt += label_dict[c]
        if c :
            txt += '['+ str(confidence) +']'
    #print("count="+str(len(pred_class)) + " txt:" + txt)
    return txt

def draw_prediction( frame, class_string ):
    x_start = frame.shape[1] -600
    cv2.putText(frame, class_string, (x_start, 75), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (255, 0, 0), 2, cv2.LINE_AA)
    return frame

def send_mail(sender_address, sender_pass, receiver_address):
    mail_content = '''Hello Customer,

There is a fire warning issued in your area please take necessary precaution and requesting you to find nearest police station for further details.

Thank You
'''
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = 'Fire Alert'
    message.attach(MIMEText(mail_content, 'plain'))
    session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_address, sender_pass) #login with mail_id and password
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()

def makeRegisteration(device_id):
    locationCode = input("Enter area code :")
    mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "",
    database = "fduc"
    )
    mycursor = mydb.cursor()
    sql = "INSERT INTO DEVICES(ID, AREA_CODE) VALUES (%s, %s)"
    val = (device_id, locationCode)
    mycursor.execute(sql,val)
    mydb.commit()
    return 1


def checkregister():
    global device_id 
    device_id = input("Enter device Id :")
    mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "",
    database = "fduc"
    )
    mycursor = mydb.cursor()
    sql = "SELECT * FROM DEVICES WHERE ID = %s"
    adr = (device_id,)
    mycursor.execute(sql,adr)
    myresult = mycursor.fetchall()
    if myresult and device_id in myresult[0]:
        return 1
    else:
        print("Device not registered do you want to continue (y/n)")
        ch = input()
        if ch == "y":
            if makeRegisteration(device_id) == 1:
                print("Registration Sucessful...")
                return 1
        else:
            print("exiting application")

def findMails(device_id):
    mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "",
    database = "fduc"
    )
    mycursor = mydb.cursor()
    sql = "SELECT EMAIL FROM CUSTOMERS WHERE DEVICE_ID = %s"
    adr = (device_id,)
    mycursor.execute(sql,adr)
    myresult = mycursor.fetchall()
    for i in myresult:
        send_mail("pmaniawasthi10@gmail.com","quvdgjoexyxzvcsm",i[0])

if checkregister() == 1:
    print("Welcome")
    predict_cam(new_model, label_dict_s, device_id = device_id)


Enter device Id :123456
Welcome


### To interface using URL

In [ ]:
predict_cam(new_model, label_dict_s, 'http://192.168.1.7:8080/video')

In [31]:
video_path = 'input/fire-detection-from-cctv/data/data/video_data/test_videos/test1.mp4'
predict ( new_model, video_path, 'test1_9.avi',  label_dict_s) 

In [23]:
send_mail("pmaniawasthi10@gmail.com","quvdgjoexyxzvcsm","awasthiprakashmani7@gmail.com")